In [1]:
import sys
print(sys.executable)


c:\Users\hncha\AppData\Local\Programs\Python\Python312\python.exe


In [17]:
# Cài đặt Thư viện và Khai báo Hàm Đọc Dữ liệu (Cell 1 Đã Cập nhật)

# ... (Giữ nguyên các import) ...
import pandas as pd
import numpy as np
import re
import csv
import io
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pickle

# --- CẤU HÌNH ---
FILE_PATHS = [
    'dict_normal.csv',
    'dict_special.csv',
    'dict_general.csv'
  
]
TRAIN_FILE = 'data_train.csv'
TARGET_STD_COLS = ['abb', 'meaning']

# **ĐÃ CẬP NHẬT:** Kiểm tra lại tất cả các delimiter phổ biến (ưu tiên dấu phẩy)
DELIMITERS = [',', ';', '\t', '|'] 
ENCODINGS = ['utf-8', 'latin-1', 'cp1252']

# --- HÀM ĐỌC FILE RẤT BỀN BỈ (ĐÃ FIX LỖI DELIMITER) ---
def robust_read_file_content(file):
    """Đọc file bằng Python native, thử các delimiter phổ biến."""
    raw_content = None
    
    # BƯỚC 1: Đọc nội dung thô (raw content)
    for enc in ENCODINGS:
        try:
            with open(file, 'r', encoding=enc) as f:
                raw_content = f.read()
                break
        except UnicodeDecodeError:
            continue
        except FileNotFoundError:
            raise FileNotFoundError(f"Không tìm thấy file: {file}. Vui lòng kiểm tra lại tên file và thư mục.")
        except Exception as e:
            continue

    if raw_content is None:
        raise ValueError(f"Không thể giải mã (decode) nội dung file {file} với bất kỳ encoding nào.")

    # BƯỚC 2: Phân tích nội dung thô bằng thư viện CSV (thử các delimiter)
    for sep in DELIMITERS:
        try:
            csv_reader = csv.reader(io.StringIO(raw_content), delimiter=sep)
            all_rows = list(csv_reader)
            if not all_rows: continue
            
            header = all_rows[0]
            data = [row for row in all_rows[1:] if len(row) >= 2]
            
            if len(data) > 10: 
                print(f"  -> Đọc thành công với delimiter: '{sep}'")
                return header, data, sep
        except Exception:
            continue
            
    raise ValueError(f"Không thể phân tích nội dung file {file} thành 2 cột với các delimiter phổ biến.")

print("Thiết lập hoàn tất. Chuyển sang Cell 2.")

Thiết lập hoàn tất. Chuyển sang Cell 2.


In [21]:
all_dictionaries = []

print("--- BƯỚC 1: ĐỌC VÀ HỢP NHẤT DỮ LIỆU TỪ ĐIỂN ---")
for file in FILE_PATHS:
    print(f"Đang xử lý file: {file}...")
    try:
        header, data, sep = robust_read_file_content(file) 
        df = pd.DataFrame(data)
        
        # Lấy header đã được dọn dẹp: 
        # 1. Chuyển thành chữ thường
        # 2. Loại bỏ khoảng trắng thừa (.strip())
        # 3. LOẠI BỎ KÝ TỰ BOM ('\ufeff')
        header_stripped = [str(c).lower().strip().replace('\ufeff', '') for c in header]
        
        found_abb_col_index = None
        found_meaning_col_index = None
        
        # Xử lý các file không có header rõ ràng (như dict_sheet2.csv)
        if file == 'dict_sheet2.csv' or all(c == '' for c in header) or len(header_stripped) < 2:
            
            if df.shape[1] >= 2:
                df = df.iloc[:, :2]
                df.columns = TARGET_STD_COLS
                print(f"  -> {file} được xử lý bằng cách lấy 2 cột đầu (dạng không header).")
            else:
                 print(f"  [LỖI CẤU TRÚC] File {file} không có đủ 2 cột dữ liệu. Bỏ qua.")
                 continue
        else:
            # Xử lý các file có header đã được làm sạch
            for i, col in enumerate(header_stripped):
                # Tên cột chính xác sau khi strip và bỏ BOM: abbreviation hoặc meaning
                if col == 'abbreviation':
                    found_abb_col_index = i
                if col == 'meaning':
                    found_meaning_col_index = i
            
            # Nếu không tìm thấy abbreviation hoặc meaning, thử các tên thay thế
            if found_abb_col_index is None or found_meaning_col_index is None:
                for i, col in enumerate(header_stripped):
                    if col in ['abb', 'viết tắt'] and found_abb_col_index is None:
                        found_abb_col_index = i
                    if col in ['origin', 'nghĩa'] and found_meaning_col_index is None:
                        found_meaning_col_index = i
            
            
            if found_abb_col_index is not None and found_meaning_col_index is not None:
                # Nếu tìm thấy, chọn và đổi tên cột
                df = df.iloc[:, [found_abb_col_index, found_meaning_col_index]]
                df.columns = TARGET_STD_COLS 
                print(f"  -> {file} xử lý thành công qua header.")
            else:
                print(f"  [LỖI HEADER] Không tìm thấy cột 'abbreviation'/'meaning' (hoặc tên thay thế) trong header file: {file}. Header được đọc (đã strip & bỏ BOM): {header_stripped}. Bỏ qua.")
                continue

        all_dictionaries.append(df) 

    except FileNotFoundError:
        print(f"  [LỖI FILE NOT FOUND] KHÔNG TÌM THẤY FILE: {file}. Vui lòng kiểm tra lại tên file và thư mục.")
    except Exception as e:
        print(f"  [LỖI CHUNG] Xử lý file {file} thất bại: {e}. Bỏ qua.")

# Hợp nhất và làm sạch dữ liệu
if all_dictionaries:
    master_dict_df = pd.concat(all_dictionaries, ignore_index=True)
    master_dict_df.dropna(subset=['abb', 'meaning'], inplace=True)
    master_dict_df['abb'] = master_dict_df['abb'].astype(str).str.lower().str.strip()
    master_dict_df['meaning'] = master_dict_df['meaning'].astype(str).str.lower().str.strip()
    master_dict_df.drop_duplicates(subset=['abb', 'meaning'], inplace=True)

    print("\n--- KẾT QUẢ HỢP NHẤT THÀNH CÔNG ---")
    print(f"Tổng số cặp viết tắt/nghĩa sau khi hợp nhất và làm sạch: {len(master_dict_df)}")
    print(master_dict_df.head())
    print("\n✅ Dữ liệu từ điển đã sẵn sàng. Vui lòng chạy Cell 3 để gán nhãn.")
else:
    print("\n[LỖI NẶNG] KHÔNG CÓ DATASET NÀO ĐƯỢC LOAD THÀNH CÔNG.")
    
    # CHẠY DÒNG NÀY ĐỂ LƯU TỪ ĐIỂN CẦN THIẾT CHO FLASK
master_dict_df.to_csv('master_dict_data.csv', index=False, encoding='utf-8')
print("Đã tạo file master_dict_data.csv thành công.")

--- BƯỚC 1: ĐỌC VÀ HỢP NHẤT DỮ LIỆU TỪ ĐIỂN ---
Đang xử lý file: dict_normal.csv...
  -> Đọc thành công với delimiter: ';'
  -> dict_normal.csv xử lý thành công qua header.
Đang xử lý file: dict_special.csv...
  -> Đọc thành công với delimiter: ';'
  -> dict_special.csv xử lý thành công qua header.
Đang xử lý file: dict_general.csv...
  -> Đọc thành công với delimiter: ';'
  -> dict_general.csv xử lý thành công qua header.

--- KẾT QUẢ HỢP NHẤT THÀNH CÔNG ---
Tổng số cặp viết tắt/nghĩa sau khi hợp nhất và làm sạch: 1685
  abb meaning
0   ă       á
1   â       á
2   ế       á
3   í       á
4  àh       à

✅ Dữ liệu từ điển đã sẵn sàng. Vui lòng chạy Cell 3 để gán nhãn.
Đã tạo file master_dict_data.csv thành công.


In [24]:
# --- BƯỚC 2: XÂY DỰNG P-LIST THỦ CÔNG ---
profanity_list = set([
    'vãi lồn', 'vãi cả lồn', 'vãi cứt', 'vãi đái', 'vãi l*n', 
    'địt', 'địt mẹ', 'đéo', 'đĩ mẹ', 'đkm', 'đậu má', 'đm', 
    'cặc', 'con cặc', 'thằng cặc', 'cứk', 'c*c', 'con đĩ', 'đĩ',
    'lồn', 'cái lồn', 'lồn', 'l*n', 
    'chó má', 'con chó', 'thằng chó', 'chó chết', 'óc chó', 'óc heo'
    'súc vật', 'chết tiệt', 'ngu lol', 'ngu vcl', 'quân phản động', 'mất dạy', 'ba que','ngu','đần'
])
print("P-List (Danh sách từ xúc phạm) đã được tạo.")

# --- BƯỚC 3: GÁN NHÃN TỰ ĐỘNG CHO TẬP HUẤN LUYỆN ---
print("\n--- BƯỚC 3: GÁN NHÃN TỰ ĐỘNG ---")

# Đọc file train
_, train_data, _ = robust_read_file_content(TRAIN_FILE)
train_df = pd.DataFrame(train_data)

# File train có cấu trúc: abb, start_index, end_index, cmt (cột 3), origin
if train_df.shape[1] >= 4:
    train_df = train_df.iloc[:, 3]
    train_df = pd.DataFrame(train_df)
    train_df.columns = ['text']
else:
    raise ValueError(f"File {TRAIN_FILE} không có đủ cột dữ liệu (cmt) cần thiết. Dừng chương trình.")

train_df['text'] = train_df['text'].astype(str).str.lower().str.strip()
norm_dict = dict(zip(master_dict_df['abb'], master_dict_df['meaning']))

def normalize_and_label(text, norm_dict, profanity_list):
    """Thực hiện chuẩn hóa (de-abbreviate) và gán nhãn."""
    normalized_text = text
    # 1. Chuẩn hóa
    for abb, meaning in norm_dict.items():
        pattern = r'\b' + re.escape(abb) + r'\b'
        normalized_text = re.sub(pattern, meaning, normalized_text)
        
    is_profane = 0
    # 2. Gán nhãn
    for profane_word in profanity_list:
        if profane_word in normalized_text:
            is_profane = 1
            break
    
    return normalized_text, is_profane

# Áp dụng hàm gán nhãn
train_df[['normalized_text', 'label']] = train_df['text'].apply(
    lambda x: pd.Series(normalize_and_label(x, norm_dict, profanity_list))
)

print(train_df['label'].value_counts())
train_df.to_csv('labeled_train_data.csv', index=False, encoding='utf-8')
print("\nĐã lưu tập dữ liệu đã gán nhãn vào: labeled_train_data.csv. Chuyển sang Cell 4.")

P-List (Danh sách từ xúc phạm) đã được tạo.

--- BƯỚC 3: GÁN NHÃN TỰ ĐỘNG ---
  -> Đọc thành công với delimiter: ';'
label
0    1004
1     213
Name: count, dtype: int64

Đã lưu tập dữ liệu đã gán nhãn vào: labeled_train_data.csv. Chuyển sang Cell 4.


In [25]:
# --- BƯỚC 4: HUẤN LUYỆN MÔ HÌNH PHÂN LOẠI LOGISTIC REGRESSION ---

df_labeled = train_df # Sử dụng DataFrame đã được gán nhãn từ Cell 3
print(f"Bắt đầu huấn luyện mô hình với {len(df_labeled)} mẫu dữ liệu.")

# 1. Chia Dữ liệu
X_train, X_test, y_train, y_test = train_test_split(
    df_labeled['normalized_text'], 
    df_labeled['label'], 
    test_size=0.2, 
    random_state=42,
    stratify=df_labeled['label']
)

# 2. Vector hóa (TF-IDF)
tfidf_vectorizer = TfidfVectorizer(
    ngram_range=(1, 3), # Sử dụng n-gram từ 1 đến 3
    max_features=10000 
)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train.astype(str))
X_test_tfidf = tfidf_vectorizer.transform(X_test.astype(str))

# 3. Huấn luyện Mô hình
model = LogisticRegression(solver='liblinear', random_state=42, class_weight='balanced') 
model.fit(X_train_tfidf, y_train)

# 4. Đánh giá Mô hình
y_pred = model.predict(X_test_tfidf)
print("\n--- BÁO CÁO ĐÁNH GIÁ MÔ HÌNH ---")
print(classification_report(y_test, y_pred))

# 5. Lưu Mô hình và Vectorizer
with open('profanity1_model.pkl', 'wb') as f:
    pickle.dump(model, f)
    
with open('tfidf_vectorizer1.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

print("\n✅ HOÀN THÀNH TẤT CẢ CÁC BƯỚC! Đã lưu mô hình và Vectorizer.")
print("Bạn có thể sử dụng profanity_model.pkl và tfidf_vectorizer.pkl để tích hợp API.")

Bắt đầu huấn luyện mô hình với 1217 mẫu dữ liệu.

--- BÁO CÁO ĐÁNH GIÁ MÔ HÌNH ---
              precision    recall  f1-score   support

           0       0.96      0.94      0.95       201
           1       0.73      0.81      0.77        43

    accuracy                           0.91       244
   macro avg       0.84      0.87      0.86       244
weighted avg       0.92      0.91      0.92       244


✅ HOÀN THÀNH TẤT CẢ CÁC BƯỚC! Đã lưu mô hình và Vectorizer.
Bạn có thể sử dụng profanity_model.pkl và tfidf_vectorizer.pkl để tích hợp API.


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset

# 1. Tải dữ liệu đã gán nhãn và làm sạch (từ Cell 3)
try:
    df = pd.read_csv('labeled_train_data.csv')
    df = df.dropna(subset=['normalized_text', 'label'])
except FileNotFoundError:
    print("Vui lòng đảm bảo file labeled_train_data.csv đã được tạo.")
    
# Chuyển nhãn về dạng int
df['label'] = df['label'].astype(int)

# 2. Tải PhoBERT Tokenizer
MODEL_NAME = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 3. Chuẩn bị tập dữ liệu (Chia và Mã hóa)
train_df, eval_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# Chuyển đổi Pandas DataFrame sang Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df[['normalized_text', 'label']])
eval_dataset = Dataset.from_pandas(eval_df[['normalized_text', 'label']])

# Hàm mã hóa (tokenize)
def tokenize_function(examples):
    # PhoBERT giới hạn ở 256 token, nhưng thường là 256
    return tokenizer(examples["normalized_text"], padding="max_length", truncation=True, max_length=256)

tokenized_train = train_dataset.map(tokenize_function, batched=True).remove_columns(["normalized_text", "__index_level_0__"])
tokenized_eval = eval_dataset.map(tokenize_function, batched=True).remove_columns(["normalized_text", "__index_level_0__"])

# Đổi tên cột
tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_eval = tokenized_eval.rename_column("label", "labels")
tokenized_train.set_format("torch")
tokenized_eval.set_format("torch")


# 4. Tải Mô hình và Định cấu hình cho Phân loại Nhị phân (num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# 5. Định nghĩa các tham số Huấn luyện
training_args = TrainingArguments(
    output_dir="./phobert_results",
    num_train_epochs=3,                     # Số lượng epoch
    per_device_train_batch_size=16,         # Kích thước batch cho huấn luyện
    per_device_eval_batch_size=16,          # Kích thước batch cho đánh giá
    warmup_steps=500,                       # Số bước làm nóng
    weight_decay=0.01,                      # Giảm trọng số
    logging_dir='./logs',
    logging_steps=50,
    evaluation_strategy="epoch",            # Đánh giá sau mỗi epoch
    save_strategy="epoch",                  # Lưu mô hình sau mỗi epoch
    load_best_model_at_end=True,            # Tải mô hình tốt nhất sau khi kết thúc
)

# 6. Định nghĩa Hàm Đánh giá (dùng cho Trainer)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        'accuracy': accuracy_score(p.label_ids, preds),
        'f1_macro': f1_score(p.label_ids, preds, average='macro'),
        'precision_macro': precision_score(p.label_ids, preds, average='macro'),
        'recall_macro': recall_score(p.label_ids, preds, average='macro'),
    }

# 7. Khởi tạo Trainer và Bắt đầu Fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("\n--- BẮT ĐẦU FINE-TUNING PHOBERT ---")
trainer.train()

# Lưu mô hình tốt nhất và tokenizer đã fine-tuned
trainer.save_model("./final_phobert_model")
tokenizer.save_pretrained("./final_phobert_model")
print("\n✅ Fine-tuning hoàn tất. Mô hình đã được lưu vào thư mục: final_phobert_model")